In [10]:
%reload_ext autoreload
%autoreload 2

import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('__file__'), '..')))

import time
from nirs4all.presets.ref_models import decon, nicon, customizable_nicon, nicon_classification
from nirs4all.presets.preprocessings import decon_set, nicon_set
from nirs4all.data_splitters import KennardStoneSplitter
from nirs4all.transformations import StandardNormalVariate as SNV, SavitzkyGolay as SG, Gaussian as GS, Derivate as  Dv
from nirs4all.transformations import Rotate_Translate as RT, Spline_X_Simplification as SXS, Random_X_Operation as RXO
from nirs4all.transformations import CropTransformer
from nirs4all.core.runner import ExperimentRunner
from nirs4all.core.config import Config

from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import KFold, RepeatedKFold, StratifiedKFold, RepeatedStratifiedKFold, ShuffleSplit, GroupKFold, StratifiedShuffleSplit, BaseCrossValidator, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import warnings
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)


model_sklearn = {
    "class": "sklearn.cross_decomposition.PLSRegression",
    "model_params": {
        "n_components": 21,
    }
}
    
finetune_pls_experiment = {
    "action": "finetune",
    "finetune_params": {
        'model_params': {
            'n_components': ('int', 5, 20),
        },
        'training_params': {},
        'tuner': 'sklearn'
    }
}

bacon_train = {"action": "train", "training_params": {"epochs": 2000, "batch_size": 500, "patience": 200, "cyclic_lr": True, "base_lr": 1e-6, "max_lr": 1e-3, "step_size": 400}}
bacon_train_short = {"action": "train", "training_params": {"epochs": 10, "batch_size": 500, "patience": 20, "cyclic_lr": True, "base_lr": 1e-6, "max_lr": 1e-3, "step_size": 40}}
bacon_finetune = {
    "action": "finetune",
    "finetune_params": {
        "n_trials": 5,
        "model_params": {
            "filters_1": [8, 16, 32, 64], 
            "filters_2": [8, 16, 32, 64], 
            "filters_3": [8, 16, 32, 64]
        }
    },
    "training_params": {
        "epochs": 10,
        "verbose":0
    }
}

full_bacon_finetune = {
    "action": "finetune",
    "training_params": {
        "epochs": 500,
        "patience": 100,
    },
    "finetune_params": {
        "nb_trials": 150,
        "model_params": {
            'spatial_dropout': (float, 0.01, 0.5),
            'filters1': [4, 8, 16, 32, 64, 128, 256],
            'kernel_size1': [3, 5, 7, 9, 11, 13, 15],
            # 'strides1': [1, 2, 3, 4, 5],
            # 'activation1': ['relu', 'selu', 'elu', 'swish'],
            'dropout_rate': (float, 0.01, 0.5),
            'filters2': [4, 8, 16, 32, 64, 128, 256],
            # 'kernel_size2': [3, 5, 7, 9, 11, 13, 15],
            # 'strides2': [1, 2, 3, 4, 5],
            'activation2': ['relu', 'selu', 'elu', 'swish'],
            'normalization_method1': ['BatchNormalization', 'LayerNormalization'],
            'filters3': [4, 8, 16, 32, 64, 128, 256],
            # 'kernel_size3': [3, 5, 7, 9, 11, 13, 15],
            # 'strides3': [1, 2, 3, 4, 5],
            'activation3': ['relu', 'selu', 'elu', 'swish'],
            # 'normalization_method2': ['BatchNormalization', 'LayerNormalization'],
            # 'dense_units': [4, 8, 16, 32, 64, 128, 256],
            'dense_activation': ['relu', 'selu', 'elu', 'swish'],
        },
        # "training_params": {
        #     "batch_size": [32, 64, 128, 256, 512],
        #     "cyclic_lr": [True, False],
        #     "base_lr": (float, 1e-6, 1e-2),
        #     "max_lr": (float, 1e-3, 1e-1),
        #     "step_size": (int, 500, 5000),
        # },
    }
}


x_pipeline_full = [
    RobustScaler(quantile_range=(0.05, 0.95)),
    {"samples": [None, None, None, None, SXS, RXO]},
    {"split": RepeatedKFold(n_splits=3, n_repeats=1)},
    {"features": [None, GS(3,1), SG, SNV, Dv, [GS, SNV], [GS, GS],[GS, SG],[SG, SNV], [GS, Dv], [SG, Dv]]},
    MinMaxScaler(feature_range=(0, 0.9), clip=False)
]


bacon_finetune_classif = {
    "action": "finetune",
    "task": "classification",
    "finetune_params": {
        "n_trials": 5,
        "model_params": {
            "filters_1": [8, 16, 32, 64], 
            "filters_2": [8, 16, 32, 64], 
            "filters_3": [8, 16, 32, 64]
        }
    },
    "training_params": {
        "epochs": 5,
        "verbose":0
    }
}

finetune_randomForestclassifier = {
    "action": "finetune",
    "task": "classification",
    "finetune_params": {
        'model_params': {
            'n_estimators': ('int', 5, 20),
        },
        'training_params': {},
        'tuner': 'sklearn'
    }
}

x_pipeline_PLS = [
    RobustScaler(),
    {"split": RepeatedKFold(n_splits=3, n_repeats=1)},
    {"features": [None, GS(2,1), SG, SNV, Dv, [GS, SNV], [GS, GS],[GS, SG],[SG, SNV], [GS, Dv], [SG, Dv]]},
    MinMaxScaler()
]
            
            
x_pipeline = [
    RobustScaler(), 
    {"split": RepeatedKFold(n_splits=3, n_repeats=1)}, 
    # bacon_set(),
    MinMaxScaler()
]

x_pipelineb = [
    RobustScaler(), 
    {"samples": [RT(6)], "balance": True},
    # {"samples": [None, RT]},
    {"split": RepeatedKFold(n_splits=3, n_repeats=1)}, 
    # {"features": [None, GS(2,1), SG, SNV, Dv, [GS, SNV], [GS, GS],[GS, SG],[SG, SNV], [GS, Dv], [SG, Dv]]},
    MinMaxScaler()
]


y_pipeline = MinMaxScaler()

seed = 123459456

dataset1 = {
    "path": "../sample_data/regression",
    "params": {}
}

# processing only
config1 = Config("../sample_data/regression", x_pipeline_full, y_pipeline, None, None, seed)
## TRAINING
# regression
config2 = Config(dataset1, x_pipeline, y_pipeline, nicon, bacon_train_short, seed)
config3 = Config(dataset1, x_pipeline_PLS, y_pipeline, PLSRegression(n_components=10), None, seed)
# classification
config4 = Config("../sample_data/classification", x_pipeline, None, nicon_classification, {"task":"classification", "training_params":{"epochs":10, "patience": 100, "verbose":0}}, seed*2)
config4b = Config("../sample_data/binary", x_pipelineb, None, nicon_classification, {"task":"classification", "training_params":{"epochs":10, "patience": 100, "verbose":0}}, seed*2)
config5 = Config("../sample_data/binary", x_pipeline, None, nicon_classification, {"task":"classification", "training_params":{"epochs":5}, "verbose":0}, seed*2)
config6 = Config("../sample_data/classification", x_pipeline, None, RandomForestClassifier, {"task":"classification"}, seed*2)
config7 = Config("../sample_data/binary", x_pipeline, None, RandomForestClassifier, {"task":"classification"}, seed*2)
## FINETUNING
# regression
config8 = Config("../sample_data/regression", x_pipeline, y_pipeline, nicon, bacon_finetune, seed)
config9 = Config("../sample_data/regression", x_pipeline, y_pipeline, model_sklearn, finetune_pls_experiment, seed)
# classification
config10 = Config("../sample_data/classification", x_pipeline, None, nicon_classification, bacon_finetune_classif, seed*2)
config10b = Config("../sample_data/binary", x_pipeline, None, nicon_classification, bacon_finetune_classif, seed*2)
config11 = Config("../sample_data/classification", x_pipelineb, None, RandomForestClassifier, finetune_randomForestclassifier, seed*2)
config11b = Config("../sample_data/binary", x_pipeline, None, RandomForestClassifier, finetune_randomForestclassifier, seed*2)


configs = [config1, config2, config3, config4, config4b, config5, config6, config7, config8, config9, config10, config10b, config11, config11b]
# configs = [config10b, config11, config11b]
# configs = [config3]
config_names = ["config1", "config2", "config3", "config4", "config4b", "config5", "config6", "config7", "config8", "config9", "config10", "config10b", "config11", "config11b"]
for i, config in enumerate(configs):
    print("#" * 20)
    print(f"Config {i}: {config_names[i]}")
    print("#" * 20)
    start = time.time()
    runner = ExperimentRunner([config], resume_mode="restart")
    datasets, predictions, scores, best_params = runner.run()
    end = time.time()
    print(f"Time elapsed: {end-start} seconds")


d:\Workspace\ML\NIRS\nirs4all\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from autonotebook import tqdm as notebook_tqdm
2025-05-13 13:44:18,513 - INFO - ================================================================================
2025-05-13 13:44:18,513 - INFO - Running config: Config(dataset='../sample_data/regression', x_pipeline=[RobustScaler(), {'samples': [None, None, None, None, <class 'nirs4all.transformations._spline_augmentation.Spline_X_Simplification'>, <class 'nirs4all.transformations._random_augmentation.Random_X_Operation'>]}, {'split': RepeatedKFold(n_repeats=1, n_splits=3, random_state=None)}, {'features': [None, Gaussian(), <class 'nirs4all.transformations._nirs.SavitzkyGolay'>, <class 'sklearn.preprocessing._data.StandardScaler'>, <class 'nirs4all.transformations._scaler.Derivate'>, [<class 'nirs4all.transformations._standard.Ga

####################
Config 0: config1
####################
>> Browsing ../sample_data/regression
No train_group file found for ../sample_data/regression.
No test_group file found for ../sample_data/regression.


2025-05-13 13:44:19,660 - INFO - Dataset(x_train:(130, 2151) - y_train:(130, 1), x_test:(59, 2151) - y_test:(59, 1))
2025-05-13 13:44:19,660 - INFO - ### PROCESSING DATASET ###


>>>>>>>>>>>>>>>>
[0.97754324 0.97708536 0.99443555 ... 1.02971265 1.01004455 1.00526501]
Augmented Data Range: -30.98248674419553 3.129359016191828


2025-05-13 13:44:20,338 - INFO - Dataset(x_train:(780, 23661) - y_train:(780, 1), x_test:(59, 23661) - y_test:(59, 1))
Folds size: 86-44, 87-43, 87-43
2025-05-13 13:44:20,339 - INFO - ### PREPARING MODEL ###
2025-05-13 13:44:20,340 - INFO - Running config > {'dataset': '../sample_data/regression', 'x_pipeline': [{'class': 'sklearn.preprocessing.RobustScaler', 'params': {'copy': True, 'quantile_range': [25.0, 75.0], 'unit_variance': False, 'with_centering': True, 'with_scaling': True}}, {'samples': [None, None, None, None, {'class': 'nirs4all.transformations.Spline_X_Simplification', 'params': None}, {'class': 'nirs4all.transformations.Random_X_Operation', 'params': None}]}, {'split': {'class': 'sklearn.model_selection.RepeatedKFold', 'params': {'cv': {'class': 'sklearn.model_selection.KFold', 'params': None}, 'n_repeats': 1, 'random_state': None, 'cvargs': {'n_splits': 3}}}}, {'features': [None, {'class': 'nirs4all.transformations.Gaussian', 'params': {'copy': True, 'order': 2, 'sigma'

...

Best model params: {'n_estimators': 17, 'num_classes': 10}
Training fold 1, with shapes: (60, 2152) (60, 1) (30, 2152) (30, 1)
Training fold 2, with shapes: (60, 2152) (60, 1) (30, 2152) (30, 1)
Training fold 3, with shapes: (60, 2152) (60, 1) (30, 2152) (30, 1)
Time elapsed: 1.804358959197998 seconds
####################
Config 13: config11b
####################
>> Browsing ../sample_data/binary
No train_group file found for ../sample_data/binary.
No test_group file found for ../sample_data/binary.


2025-05-13 13:46:08,538 - INFO - Dataset(x_train:(300, 1665) - y_train:(300, 1), x_test:(129, 1665) - y_test:(129, 1))
2025-05-13 13:46:08,538 - INFO - ### PROCESSING DATASET ###
2025-05-13 13:46:08,683 - INFO - Dataset(x_train:(300, 1665) - y_train:(300, 1), x_test:(129, 1665) - y_test:(129, 1))
Folds size: 200-100, 200-100, 200-100
2025-05-13 13:46:08,684 - INFO - ### PREPARING MODEL ###
2025-05-13 13:46:08,686 - INFO - Running config > {'dataset': '../sample_data/binary', 'x_pipeline': [{'class': 'sklearn.preprocessing.RobustScaler', 'params': {'copy': True, 'quantile_range': [25.0, 75.0], 'unit_variance': False, 'with_centering': True, 'with_scaling': True}}, {'split': {'class': 'sklearn.model_selection.RepeatedKFold', 'params': {'cv': {'class': 'sklearn.model_selection.KFold', 'params': None}, 'n_repeats': 1, 'random_state': None, 'cvargs': {'n_splits': 3}}}}, {'class': 'sklearn.preprocessing.MinMaxScaler', 'params': {'clip': False, 'copy': True, 'feature_range': [0, 1]}}], 'y_pip

Using framework: sklearn


2025-05-13 13:46:09,356 - INFO - Evaluation Metrics fold_0: {'accuracy': 0.9534883720930233}
2025-05-13 13:46:09,357 - INFO - Evaluation Metrics fold_1: {'accuracy': 0.9534883720930233}
2025-05-13 13:46:09,358 - INFO - Evaluation Metrics fold_2: {'accuracy': 0.9457364341085271}
2025-05-13 13:46:09,359 - INFO - Evaluation Metrics mean: {'accuracy': 0.9534883720930233}
2025-05-13 13:46:09,359 - INFO - Evaluation Metrics best: {'accuracy': 0.9534883720930233}
2025-05-13 13:46:09,360 - INFO - Evaluation Metrics weighted: {'accuracy': 0.9534883720930233}
2025-05-13 13:46:09,362 - INFO - Metrics saved to results\sample_databinary\RandomForestClassifier\experiment_c49bf09a\metrics.json
2025-05-13 13:46:09,363 - INFO - Best parameters {'n_estimators': 9, 'num_classes': 2} saved to results\sample_databinary\RandomForestClassifier\experiment_c49bf09a\best_params.json
2025-05-13 13:46:09,365 - INFO - Predictions saved to results\sample_databinary\RandomForestClassifier\experiment_c49bf09a\predict

Best model params: {'n_estimators': 9, 'num_classes': 2}
Training fold 1, with shapes: (200, 1665) (200, 1) (100, 1665) (100, 1)
Training fold 2, with shapes: (200, 1665) (200, 1) (100, 1665) (100, 1)
Training fold 3, with shapes: (200, 1665) (200, 1) (100, 1665) (100, 1)
Time elapsed: 1.683943271636963 seconds
